# Modules

In [1]:
import os
import pyspark


from pyspark.sql.functions import *
from pyspark.sql.types import *

# Constantes

In [2]:
PATH_DATA = r"C:\Users\ahoun\Documents\Documents\Nuku Data\ETL-real-estate-prices\Data"
PATH_DATA_RAW = rf"{PATH_DATA}\Raw"
PATH_DATA_TRANSFORMED = rf"{PATH_DATA}\Clean"

# Processing

In [3]:
spark = pyspark.sql.SparkSession.builder \
    .appName("Real_state") \
    .getOrCreate()

In [4]:
spark.active()

In [5]:
df = spark.read.json(rf'{PATH_DATA_RAW}\data_page_1.json')
df.printSchema()

root
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- price: string (nullable = true)
 |-- room: string (nullable = true)
 |-- surface: string (nullable = true)
 |-- title: string (nullable = true)



In [6]:
df.show()

+--------------------+---+-------+----+-------+--------------------+
|         description| id|  price|room|surface|               title|
+--------------------+---+-------+----+-------+--------------------+
|Les maisons de LY...|  0|   NULL| 5.5|    220|«Maison contiguë ...|
|Im idyllischen Ae...|  1|1515120| 6.5|    250|«Neue Doppeleinfa...|
|Das freistehende ...|  2|1348200| 5.5|    190|«Neubau Einfamili...|
|Das freistehende ...|  3|1348200| 5.5|    190|«Neubau Einfamili...|
|Nouvelle promotio...|  4|1784760| 5.5|    210|«Exigez le meille...|
|Nouvelle promotio...|  5|1861800| 5.5|    210|«Le meilleur pour...|
|Nouvelle promotio...|  6|1784760| 5.5|    210|«À la hauteur de ...|
|Nouvelle promotio...|  7|1784760| 5.5|    210|«Une occasion à n...|
|Nouvelle promotio...|  8|1861800| 5.5|    210|«Une opportunité ...|
|Érigé sur une jol...|  9|1181280| 5.5|    180|«Environnement id...|
|Projekt In dieser...| 10|1919580| 5.5|    260|«RÄBBERG - AUSSIC...|
|Cette nouvelle pr...| 11|1206960|

In [7]:
df = df.withColumn("id", df["id"].cast(IntegerType())) \
    .withColumn("price", df["price"].cast(IntegerType())) \
    .withColumn("room", df["room"].cast(DoubleType())) \
    .withColumn("surface", df["surface"].cast(DoubleType()))
df.printSchema()
df.show()

root
 |-- description: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- room: double (nullable = true)
 |-- surface: double (nullable = true)
 |-- title: string (nullable = true)

+--------------------+---+-------+----+-------+--------------------+
|         description| id|  price|room|surface|               title|
+--------------------+---+-------+----+-------+--------------------+
|Les maisons de LY...|  0|   NULL| 5.5|  220.0|«Maison contiguë ...|
|Im idyllischen Ae...|  1|1515120| 6.5|  250.0|«Neue Doppeleinfa...|
|Das freistehende ...|  2|1348200| 5.5|  190.0|«Neubau Einfamili...|
|Das freistehende ...|  3|1348200| 5.5|  190.0|«Neubau Einfamili...|
|Nouvelle promotio...|  4|1784760| 5.5|  210.0|«Exigez le meille...|
|Nouvelle promotio...|  5|1861800| 5.5|  210.0|«Le meilleur pour...|
|Nouvelle promotio...|  6|1784760| 5.5|  210.0|«À la hauteur de ...|
|Nouvelle promotio...|  7|1784760| 5.5|  210.0|«Une occasion à n...|
|Nouvel

In [8]:
json_files = [os.path.join(PATH_DATA_RAW, f) for f in os.listdir(PATH_DATA_RAW) if f.endswith(".json")]

# Initialisez un DataFrame vide
concatenated_df = None

# Parcourez la liste des fichiers JSON et lisez-les, puis concaténez-les
for json_file in json_files:
    # Lisez chaque fichier JSON dans un DataFrame
    temp_df = spark.read.json(json_file)
    
    # Si le DataFrame temporaire n'est pas vide, concaténez-le
    if not temp_df.isEmpty():
        if concatenated_df is None:
            concatenated_df = temp_df
        else:
            concatenated_df = concatenated_df.union(temp_df)

# Si "concatenated_df" n'est pas vide, affichez le schéma et les données
if concatenated_df is not None and not concatenated_df.isEmpty():
    concatenated_df.printSchema()
    concatenated_df.show()

root
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- price: string (nullable = true)
 |-- room: string (nullable = true)
 |-- surface: string (nullable = true)
 |-- title: string (nullable = true)

+--------------------+---+-------+----+-------+--------------------+
|         description| id|  price|room|surface|               title|
+--------------------+---+-------+----+-------+--------------------+
|Les maisons de LY...|  0|   NULL| 5.5|    220|«Maison contiguë ...|
|Im idyllischen Ae...|  1|1515120| 6.5|    250|«Neue Doppeleinfa...|
|Das freistehende ...|  2|1348200| 5.5|    190|«Neubau Einfamili...|
|Das freistehende ...|  3|1348200| 5.5|    190|«Neubau Einfamili...|
|Nouvelle promotio...|  4|1784760| 5.5|    210|«Exigez le meille...|
|Nouvelle promotio...|  5|1861800| 5.5|    210|«Le meilleur pour...|
|Nouvelle promotio...|  6|1784760| 5.5|    210|«À la hauteur de ...|
|Nouvelle promotio...|  7|1784760| 5.5|    210|«Une occasion à n...|
|Nouvelle p

In [9]:
concatenated_df.count()

1008